In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import torch
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from collections import Counter, defaultdict
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import (
    GCNConv, GATConv, SAGEConv, GINConv,
    global_mean_pool, global_max_pool, global_add_pool,
    BatchNorm, LayerNorm
)

In [2]:
EEG_Dataset = pd.read_csv(r"EEG_Full_Spectrum_Features_All_Users.csv")
print(EEG_Dataset)

       User  Page  Product  Bought  Electrode  Delta_Freq_1_FFT  \
0       S01     1        1       0          1        243.347126   
1       S01     1        1       0          2        143.530647   
2       S01     1        1       0          3        447.828155   
3       S01     1        1       0          4        309.990625   
4       S01     1        1       0          5         63.902792   
...     ...   ...      ...     ...        ...               ...   
112798  S44     6       24       0         15        192.445131   
112799  S44     6       24       0         16        335.046856   
112800  S44     6       24       0         17        403.756075   
112801  S44     6       24       0         18         77.173027   
112802  S44     6       24       0         19        125.191184   

        Delta_Freq_1_PSD  Delta_Freq_2_FFT  Delta_Freq_2_PSD  \
0               1.375451        339.230580          1.291068   
1               1.042604        333.600483          0.998860   
2  

In [3]:
EEG_Dataset = EEG_Dataset.drop(columns=['Unnamed: 103'])
print(EEG_Dataset.isnull().sum())

User                 0
Page                 0
Product              0
Bought               0
Electrode            0
                    ..
Gamma_Freq_43_PSD    0
Gamma_Freq_44_FFT    0
Gamma_Freq_44_PSD    0
Gamma_Freq_45_FFT    0
Gamma_Freq_45_PSD    0
Length: 103, dtype: int64


In [4]:
EEG_Dataset['product_page'] = EEG_Dataset['Page'].astype(str) + EEG_Dataset['Product'].astype(str)
EEG_Dataset['product_page'] = EEG_Dataset['product_page'].astype(int)
print(EEG_Dataset)

       User  Page  Product  Bought  Electrode  Delta_Freq_1_FFT  \
0       S01     1        1       0          1        243.347126   
1       S01     1        1       0          2        143.530647   
2       S01     1        1       0          3        447.828155   
3       S01     1        1       0          4        309.990625   
4       S01     1        1       0          5         63.902792   
...     ...   ...      ...     ...        ...               ...   
112798  S44     6       24       0         15        192.445131   
112799  S44     6       24       0         16        335.046856   
112800  S44     6       24       0         17        403.756075   
112801  S44     6       24       0         18         77.173027   
112802  S44     6       24       0         19        125.191184   

        Delta_Freq_1_PSD  Delta_Freq_2_FFT  Delta_Freq_2_PSD  \
0               1.375451        339.230580          1.291068   
1               1.042604        333.600483          0.998860   
2  

In [5]:
EEG_Dataset = EEG_Dataset.drop(columns=['Product', 'Page'])
EEG_Dataset = EEG_Dataset[[EEG_Dataset.columns[0], 'product_page'] + EEG_Dataset.columns.drop(['product_page', EEG_Dataset.columns[0]]).tolist()]
print(EEG_Dataset)

       User  product_page  Bought  Electrode  Delta_Freq_1_FFT  \
0       S01            11       0          1        243.347126   
1       S01            11       0          2        143.530647   
2       S01            11       0          3        447.828155   
3       S01            11       0          4        309.990625   
4       S01            11       0          5         63.902792   
...     ...           ...     ...        ...               ...   
112798  S44           624       0         15        192.445131   
112799  S44           624       0         16        335.046856   
112800  S44           624       0         17        403.756075   
112801  S44           624       0         18         77.173027   
112802  S44           624       0         19        125.191184   

        Delta_Freq_1_PSD  Delta_Freq_2_FFT  Delta_Freq_2_PSD  \
0               1.375451        339.230580          1.291068   
1               1.042604        333.600483          0.998860   
2              

In [6]:
electrode_columns = [col for col in EEG_Dataset.columns if col not in ['User', 'product_page', 'Bought', 'Electrode']]

df_pivoted = EEG_Dataset.pivot_table(index=['User', 'product_page', 'Bought'], columns='Electrode', values=electrode_columns, aggfunc='first')

df_pivoted.columns = [f"{col[0]}_Electrode_{col[1]}" for col in df_pivoted.columns]

df_pivoted = df_pivoted.reset_index()

print(df_pivoted)

     User  product_page  Bought  Alpha_Freq_10_FFT_Electrode_1  \
0     S01            11       0                     389.347719   
1     S01            12       0                      17.319813   
2     S01            13       0                     102.179924   
3     S01            14       1                      50.225631   
4     S01            15       0                      10.605075   
...   ...           ...     ...                            ...   
3247  S44           619       0                      61.262399   
3248  S44           620       1                      19.770211   
3249  S44           621       0                      38.829805   
3250  S44           622       0                     172.130770   
3251  S44           624       0                      25.977570   

      Alpha_Freq_10_FFT_Electrode_2  Alpha_Freq_10_FFT_Electrode_3  \
0                        291.331736                     271.866244   
1                          7.056958                      10.668018 

In [7]:
df_pivoted.to_csv(r'EEG_Full_Spectrum_Features_All_Users1_pivoted.csv')

In [8]:
df_pivoted.head(5)

,User,product_page,Bought,Alpha_Freq_10_FFT_Electrode_1,Alpha_Freq_10_FFT_Electrode_2,Alpha_Freq_10_FFT_Electrode_3,Alpha_Freq_10_FFT_Electrode_4,Alpha_Freq_10_FFT_Electrode_5,Alpha_Freq_10_FFT_Electrode_6,Alpha_Freq_10_FFT_Electrode_7,...,Theta_Freq_8_PSD_Electrode_10,Theta_Freq_8_PSD_Electrode_11,Theta_Freq_8_PSD_Electrode_12,Theta_Freq_8_PSD_Electrode_13,Theta_Freq_8_PSD_Electrode_14,Theta_Freq_8_PSD_Electrode_15,Theta_Freq_8_PSD_Electrode_16,Theta_Freq_8_PSD_Electrode_17,Theta_Freq_8_PSD_Electrode_18,Theta_Freq_8_PSD_Electrode_19
0,S01,11,0,389.347719,291.331736,271.866244,370.012184,294.588292,163.023913,275.125584,...,0.858478,0.800493,0.382462,0.471643,0.522279,0.871569,0.655299,0.467571,0.468216,0.254179
1,S01,12,0,17.319813,7.056958,10.668018,14.129221,22.184167,12.976338,21.951340,...,0.105492,0.378344,1.103498,0.730954,1.533291,2.569766,0.194050,0.558652,0.486367,0.229822
2,S01,13,0,102.179924,139.136277,107.440642,67.700625,154.911032,71.797798,82.641631,...,1.021204,1.066130,0.233821,0.313982,0.407564,1.123489,0.810551,0.694516,0.598272,0.151814
3,S01,14,1,50.225631,7.532174,8.082533,29.022488,59.281816,58.328360,53.434501,...,0.508811,0.466100,0.570128,1.472353,2.988221,2.222447,0.258775,0.592465,1.613849,0.468974
4,S01,15,0,10.605075,28.317680,14.632045,11.895777,35.467673,18.639482,23.225534,...,0.169072,0.188846,0.541452,0.433949,1.281652,1.235423,0.723745,0.242880,0.946793,0.209386


Graph Neural Networks

In [3]:
class BaselineGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super(BaselineGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.dropout = nn.Dropout(p=dropout)
        self.lin = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        return self.lin(x)

class BaselineGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads=4, dropout=0.5):
        super(BaselineGAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=num_heads, concat=True)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels, heads=1, concat=True)
        self.dropout = nn.Dropout(p=dropout)
        self.lin = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.elu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = F.elu(self.conv2(x, edge_index))
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        return self.lin(x)

class BaselineSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super(BaselineSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.dropout = nn.Dropout(p=dropout)
        self.lin = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        return self.lin(x)

class ResidualGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super(ResidualGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.skip_proj = nn.Linear(in_channels, hidden_channels) if in_channels != hidden_channels else nn.Identity()
        self.dropout = nn.Dropout(p=dropout)
        self.lin = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        # First GCN layer
        h1 = F.relu(self.conv1(x, edge_index))
        h1 = self.dropout(h1)
        skip = self.skip_proj(x)
        h2 = self.conv2(h1, edge_index)
        h2 = self.dropout(h2)
        h = F.relu(h2 + skip)
        h = global_mean_pool(h, batch)
        return self.lin(h)

class HybridModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads=4, dropout=0.5):
        super(HybridModel, self).__init__()
        self.mlp1 = nn.Linear(in_channels, hidden_channels)
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.gcn1 = GCNConv(hidden_channels, hidden_channels)
        self.gat = GATConv(hidden_channels, hidden_channels, heads=num_heads, concat=True)
        self.gcn2 = GCNConv(hidden_channels * num_heads, hidden_channels)
        self.dropout = nn.Dropout(p=dropout)
        self.mlp2 = nn.Linear(hidden_channels, out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.bn1(self.mlp1(x)))
        x = self.dropout(x)
        x = F.relu(self.gcn1(x, edge_index))
        x = self.dropout(x)
        x = F.elu(self.gat(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.gcn2(x, edge_index))
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        x = self.bn2(self.mlp2(x))
        return x

        
class RegularizedGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super(RegularizedGNN, self).__init__()
        self.input_dropout = nn.Dropout(dropout)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.norm1 = LayerNorm(hidden_channels)
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = GATConv(hidden_channels, hidden_channels, heads=1)
        self.norm2 = LayerNorm(hidden_channels)
        self.dropout2 = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels // 2, out_channels)
        )
        
    def forward(self, x, edge_index, batch):
        x = self.input_dropout(x)
        x = self.conv1(x, edge_index)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x, edge_index)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = global_mean_pool(x, batch)
        x = self.classifier(x)
        return x

class LightweightGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super(LightweightGCN, self).__init__()
        self.conv = GCNConv(in_channels, hidden_channels)
        self.norm = LayerNorm(hidden_channels)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_channels, out_channels)
        
    def forward(self, x, edge_index, batch):
        x = self.conv(x, edge_index)
        x = self.norm(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        x = self.classifier(x)
        return x

class BalancedGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super(BalancedGAT, self).__init__()
        self.proj = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.conv = GATConv(hidden_channels, hidden_channels, heads=1)
        self.norm = LayerNorm(hidden_channels)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, out_channels)
        )
        
    def forward(self, x, edge_index, batch):
        x = self.proj(x)
        x = self.conv(x, edge_index)
        x = self.norm(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        x = self.classifier(x)
        return x

class MultiGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads=4, dropout=0.3):
        super(MultiGNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.gcn = GCNConv(hidden_channels, hidden_channels)
        self.gat = GATConv(hidden_channels, hidden_channels // num_heads, heads=num_heads)
        self.sage = SAGEConv(hidden_channels, hidden_channels)
        self.norm1 = LayerNorm(hidden_channels)
        self.norm2 = LayerNorm(hidden_channels)
        self.norm3 = LayerNorm(hidden_channels)
        self.skip_proj = nn.Linear(hidden_channels, hidden_channels)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels * 3, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels // 2, out_channels)
        )
        
    def forward(self, x, edge_index, batch):
        x = self.encoder(x)
        gcn_out = self.gcn(x, edge_index)
        gcn_out = self.norm1(gcn_out)
        gcn_out = F.relu(gcn_out)
        gat_out = self.gat(x, edge_index)
        gat_out = self.norm2(gat_out)
        gat_out = F.relu(gat_out)
        sage_out = self.sage(x, edge_index)
        sage_out = self.norm3(sage_out)
        sage_out = F.relu(sage_out)
        skip = self.skip_proj(x)
        gcn_out = gcn_out + skip
        gat_out = gat_out + skip
        sage_out = sage_out + skip
        gcn_pooled = global_mean_pool(gcn_out, batch)
        gat_pooled = global_max_pool(gat_out, batch)
        sage_pooled = global_add_pool(sage_out, batch)
        combined = torch.cat([gcn_pooled, gat_pooled, sage_pooled], dim=1)
        out = self.classifier(combined)
        return out

class ResidualAttentionGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads=4, dropout=0.3):
        super(ResidualAttentionGNN, self).__init__()
        self.input_transform = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.gat1 = GATConv(hidden_channels, hidden_channels // num_heads, heads=num_heads)
        self.gat2 = GATConv(hidden_channels, hidden_channels // num_heads, heads=num_heads)
        self.norm1 = LayerNorm(hidden_channels)
        self.norm2 = LayerNorm(hidden_channels)
        self.attention = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.Tanh(),
            nn.Linear(hidden_channels // 2, 1)
        )
        self.output = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels // 2, out_channels)
        )
        
    def forward(self, x, edge_index, batch):
        h = self.input_transform(x)
        h1 = self.gat1(h, edge_index)
        h1 = self.norm1(h1)
        h1 = F.relu(h1)
        h1 = h1 + h  
        h2 = self.gat2(h1, edge_index)
        h2 = self.norm2(h2)
        h2 = F.relu(h2)
        h2 = h2 + h1  
        attention_weights = self.attention(h2)
        attention_weights = F.softmax(attention_weights, dim=0)
        h2 = h2 * attention_weights
        pooled = global_mean_pool(h2, batch)
        out = self.output(pooled)
        return out

class DeepGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, dropout=0.3):
        super(DeepGNN, self).__init__()
        self.input_proj = nn.Linear(in_channels, hidden_channels)
        self.gnn_layers = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.skip_connections = nn.ModuleList()
        
        for i in range(num_layers):
            self.gnn_layers.append(nn.ModuleList([
                GCNConv(hidden_channels, hidden_channels),
                GATConv(hidden_channels, hidden_channels // 4, heads=4),
                SAGEConv(hidden_channels, hidden_channels)
            ]))
            self.norms.append(nn.ModuleList([
                LayerNorm(hidden_channels),
                LayerNorm(hidden_channels),
                LayerNorm(hidden_channels)
            ]))
            if i > 0:
                self.skip_connections.append(nn.Linear(hidden_channels, hidden_channels))
        
        self.output = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels // 2, out_channels)
        )
        
    def forward(self, x, edge_index, batch):
        h = self.input_proj(x)
        
        for i, (gnn_layer, norm_layer) in enumerate(zip(self.gnn_layers, self.norms)):
            gcn_out = gnn_layer[0](h, edge_index)
            gat_out = gnn_layer[1](h, edge_index)
            sage_out = gnn_layer[2](h, edge_index)
            
            gcn_out = F.relu(norm_layer[0](gcn_out))
            gat_out = F.relu(norm_layer[1](gat_out))
            sage_out = F.relu(norm_layer[2](sage_out))
            
            h_new = (gcn_out + gat_out + sage_out) / 3
            
            if i > 0:
                h_new = h_new + self.skip_connections[i-1](h)
            
            h = h_new
        
        pooled = global_mean_pool(h, batch)
        out = self.output(pooled)
        return out 

In [8]:
def calculate_class_weights(labels):
    """Calculate class weights for imbalanced dataset"""
    class_counts = Counter(labels)
    total_samples = len(labels)
    weights = {cls: total_samples / (len(class_counts) * count) 
              for cls, count in class_counts.items()}
    return torch.tensor([weights[i] for i in range(len(class_counts))])


def evaluate_model(model, data_loader, device, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)
            total_loss += loss.item() * data.num_graphs
            
            pred = out.argmax(dim=1)
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision_per_class = precision_score(all_labels, all_preds, average=None, zero_division=0)
    recall_per_class = recall_score(all_labels, all_preds, average=None, zero_division=0)
    f1_per_class = f1_score(all_labels, all_preds, average=None, zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    
    return {
        'loss': total_loss / len(data_loader.dataset),
        'accuracy': accuracy,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class,
        'confusion_matrix': cm
    }

def print_class_metrics(results, model_name, fold):
    print(f"\nDetailed Class Metrics for {model_name} - Fold {fold + 1}:")
    print("=" * 50)
    print(f"Overall Accuracy: {results['accuracy']:.4f}")
    print("\nPer-Class Metrics:")
    print("-" * 30)
    for i in range(len(results['precision_per_class'])):
        print(f"Class {i}:")
        print(f"  Precision: {results['precision_per_class'][i]:.4f}")
        print(f"  Recall:    {results['recall_per_class'][i]:.4f}")
        print(f"  F1 Score:  {results['f1_per_class'][i]:.4f}")
    print("\nConfusion Matrix:")
    print(results['confusion_matrix'])
    print("=" * 50)


def train_and_evaluate(model_class, model_name, train_graphs, test_graphs, device, epochs=100, patience=15):
    graphs = [item['graph'] for item in train_graphs]
    labels = [item['label'] for item in train_graphs]
    for g, y in zip(graphs, labels):
        g.y = torch.tensor([y], dtype=torch.long)

    test_graphs_copy = []
    for item in test_graphs:
        g = item['graph']
        g.y = torch.tensor([item['label']], dtype=torch.long)
        test_graphs_copy.append(g)
    test_loader_full = DataLoader(test_graphs_copy, batch_size=32, shuffle=False)
    true_test_labels = []
    with torch.no_grad():
        for data in test_loader_full:
            true_test_labels.extend(data.y.cpu().numpy())
    true_test_labels = np.array(true_test_labels)

    class_weights = calculate_class_weights(labels).to(device)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    fold_predictions = []  

    for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(graphs)), labels)):
        print(f"\n===== {model_name} - Fold {fold + 1} =====")

        train_subset = [graphs[i] for i in train_idx]
        val_subset = [graphs[i] for i in val_idx]

        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_graphs_copy, batch_size=32, shuffle=False)

        in_channels = graphs[0].x.size(1)
        out_channels = len(set(labels))
        model = model_class(in_channels=in_channels, hidden_channels=32, out_channels=out_channels, dropout=0.5).to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)
        criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

        best_val_loss = float('inf')
        best_model_state = None
        epochs_no_improve = 0

        for epoch in range(epochs):
            model.train()
            total_loss = 0
            correct = 0
            total = 0

            for data in train_loader:
                data = data.to(device)
                optimizer.zero_grad()
                out = model(data.x, data.edge_index, data.batch)
                loss = criterion(out, data.y)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                total_loss += loss.item() * data.num_graphs
                pred = out.argmax(dim=1)
                correct += (pred == data.y).sum().item()
                total += data.num_graphs

            train_loss = total_loss / len(train_loader.dataset)
            train_acc = correct / total

            val_results = evaluate_model(model, val_loader, device, criterion)
            scheduler.step(val_results['accuracy'])

            print(f"Epoch {epoch+1:02d}: Train Loss={train_loss:.4f}, Acc={train_acc:.4f} | "
                  f"Val Loss={val_results['loss']:.4f}, Acc={val_results['accuracy']:.4f}")

            if val_results['loss'] < best_val_loss:
                best_val_loss = val_results['loss']
                best_model_state = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break

        model.load_state_dict(best_model_state)
        model.eval()
        preds_fold = []
        with torch.no_grad():
            for data in test_loader:
                data = data.to(device)
                out = model(data.x, data.edge_index, data.batch)
                preds = out.argmax(dim=1)
                preds_fold.extend(preds.cpu().numpy())

        preds_fold = np.array(preds_fold)
        fold_predictions.append(preds_fold)

        fold_acc = accuracy_score(true_test_labels, preds_fold)
        fold_prec = precision_score(true_test_labels, preds_fold, average='weighted', zero_division=0)
        fold_rec = recall_score(true_test_labels, preds_fold, average='weighted', zero_division=0)
        fold_f1 = f1_score(true_test_labels, preds_fold, average='weighted', zero_division=0)

        print(f"\nFold {fold + 1} Test Metrics:")
        print(f"  Accuracy:  {fold_acc:.4f}")
        print(f"  Precision: {fold_prec:.4f}")
        print(f"  Recall:    {fold_rec:.4f}")
        print(f"  F1 Score:  {fold_f1:.4f}")
        print(f"\nFold {fold + 1} Test Classification Report:")
        print(classification_report(true_test_labels, preds_fold, digits=4))

    pred_matrix = np.vstack(fold_predictions)           
    def majority_vote_col(col):
        return np.bincount(col).argmax()
    majority_preds = np.apply_along_axis(majority_vote_col, 0, pred_matrix)

    final_acc = accuracy_score(true_test_labels, majority_preds)
    final_prec = precision_score(true_test_labels, majority_preds, average='weighted', zero_division=0)
    final_rec = recall_score(true_test_labels, majority_preds, average='weighted', zero_division=0)
    final_f1 = f1_score(true_test_labels, majority_preds, average='weighted', zero_division=0)

    print(f"\n===== Final Combined Results (Majority Vote) for {model_name} =====")
    print(f"Final Accuracy:  {final_acc:.4f}")
    print(f"Final Precision: {final_prec:.4f}")
    print(f"Final Recall:    {final_rec:.4f}")
    print(f"Final F1 Score:  {final_f1:.4f}")
    print(f"\nFinal Classification Report:")
    print(classification_report(true_test_labels, majority_preds, digits=4))

    return majority_preds.tolist(), true_test_labels.tolist()

def main():
    print("\n=== Loading Data ===")
    graph_data = torch.load(r"eeg_graphs_ordered.pt", weights_only=False)
    Xtrain = pd.read_csv(r"X_train.csv")
    train_indices = set(Xtrain['index'])

    train_graphs = [graph_data[i] for i in train_indices]
    test_graphs = [graph_data[i] for i in range(len(graph_data)) if i not in train_indices]

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    models = {
        'BaselineGCN': BaselineGCN,
        'BaselineGAT': BaselineGAT,
        'BaselineSAGE': BaselineSAGE,
        'ResidualGCN': ResidualGCN,
        'HybridModel': HybridModel,
        'RegularizedGNN': RegularizedGNN,
        'LightweightGCN': LightweightGCN,
        'BalancedGAT': BalancedGAT,
        'MultiGNN': MultiGNN,
        'ResidualAttentionGNN': ResidualAttentionGNN,
        'DeepGNN': DeepGNN
    }

    results = {}
    summary_results = []
    
    print("\n" + "="*80)
    print("SUMMARY OF ALL MODELS - MAJORITY VOTE RESULTS")
    print("="*80)
    
    for model_name, model_class in models.items():
        print(f"\n=== Testing {model_name} Model ===")
        preds_majority, trues = train_and_evaluate(model_class, model_name, train_graphs, test_graphs, device)
        results[model_name] = {
            'majority_preds': preds_majority,
            'true': trues
        }

        acc = accuracy_score(trues, preds_majority)
        prec = precision_score(trues, preds_majority, average='weighted', zero_division=0)
        rec = recall_score(trues, preds_majority, average='weighted', zero_division=0)
        f1 = f1_score(trues, preds_majority, average='weighted', zero_division=0)
        report = classification_report(trues, preds_majority, digits=4, zero_division=0)

        print(f"\n{model_name} - Final Majority Vote Results:")
        print(f"  Accuracy:  {acc:.4f}")
        print(f"  Precision: {prec:.4f}")
        print(f"  Recall:    {rec:.4f}")
        print(f"  F1 Score:  {f1:.4f}")

        summary_results.append({
            "Model": model_name,
            "Voting": "majority_5fold",
            "Test Accuracy": acc,
            "Test Precision": prec,
            "Test Recall": rec,
            "Test F1": f1,
            "Classification Report": report
        })

    print("\n" + "="*80)
    print("FINAL SUMMARY TABLE - ALL MODELS")
    print("="*80)
    print(f"{'Model':<25} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
    print("-" * 80)
    for result in summary_results:
        print(f"{result['Model']:<25} {result['Test Accuracy']:<10.4f} {result['Test Precision']:<10.4f} {result['Test Recall']:<10.4f} {result['Test F1']:<10.4f}")
    print("="*80)

    df_summary = pd.DataFrame(summary_results)
    df_summary.to_csv("GNN_Models_Summary.csv", index=False)
    print(f"\nResults saved to GNN_Models_Summary.csv")
    print(f"Total models evaluated: {len(summary_results)}")

if __name__ == "__main__":
    main()



=== Loading Data ===
Using device: cuda

SUMMARY OF ALL MODELS - MAJORITY VOTE RESULTS

=== Testing BaselineGCN Model ===

===== BaselineGCN - Fold 1 =====


C:\Users\USER\anaconda3\envs\newenv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 01: Train Loss=5.0702, Acc=0.5351 | Val Loss=1.5738, Acc=0.4146
Epoch 02: Train Loss=1.2840, Acc=0.5298 | Val Loss=0.9197, Acc=0.6334
Epoch 03: Train Loss=0.8571, Acc=0.5764 | Val Loss=0.7472, Acc=0.6449
Epoch 04: Train Loss=0.7218, Acc=0.6332 | Val Loss=0.6998, Acc=0.6104
Epoch 05: Train Loss=0.6950, Acc=0.6077 | Val Loss=0.6772, Acc=0.6910
Epoch 06: Train Loss=0.6798, Acc=0.6394 | Val Loss=0.6740, Acc=0.6641
Epoch 07: Train Loss=0.6745, Acc=0.6505 | Val Loss=0.6915, Acc=0.7063
Epoch 08: Train Loss=0.6782, Acc=0.6692 | Val Loss=0.6955, Acc=0.5605
Epoch 09: Train Loss=0.6758, Acc=0.6606 | Val Loss=0.6884, Acc=0.6814
Epoch 10: Train Loss=0.6790, Acc=0.6659 | Val Loss=0.6906, Acc=0.6679
Epoch 11: Train Loss=0.6661, Acc=0.6885 | Val Loss=0.6900, Acc=0.7351
Epoch 12: Train Loss=0.6666, Acc=0.6841 | Val Loss=0.6706, Acc=0.6756
Epoch 13: Train Loss=0.6648, Acc=0.6505 | Val Loss=0.6765, Acc=0.6142
Epoch 14: Train Loss=0.6611, Acc=0.6745 | Val Loss=0.6801, Acc=0.7121
Epoch 15: Train Loss

C:\Users\USER\anaconda3\envs\newenv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 01: Train Loss=5.6403, Acc=0.5615 | Val Loss=1.6359, Acc=0.6327
Epoch 02: Train Loss=1.6237, Acc=0.5817 | Val Loss=0.9230, Acc=0.5308
Epoch 03: Train Loss=0.9117, Acc=0.5659 | Val Loss=0.7067, Acc=0.5500
Epoch 04: Train Loss=0.7451, Acc=0.5808 | Val Loss=0.7023, Acc=0.7173
Epoch 05: Train Loss=0.6840, Acc=0.6279 | Val Loss=0.6842, Acc=0.6942
Epoch 06: Train Loss=0.6997, Acc=0.6562 | Val Loss=0.6803, Acc=0.6538
Epoch 07: Train Loss=0.6869, Acc=0.6404 | Val Loss=0.6838, Acc=0.7692
Epoch 08: Train Loss=0.6748, Acc=0.6611 | Val Loss=0.6909, Acc=0.5635
Epoch 09: Train Loss=0.6785, Acc=0.6813 | Val Loss=0.6847, Acc=0.5481
Epoch 10: Train Loss=0.6722, Acc=0.6361 | Val Loss=0.6766, Acc=0.6365
Epoch 11: Train Loss=0.6663, Acc=0.6755 | Val Loss=0.6812, Acc=0.6692
Epoch 12: Train Loss=0.6708, Acc=0.6697 | Val Loss=0.7080, Acc=0.6250
Epoch 13: Train Loss=0.6623, Acc=0.6981 | Val Loss=0.6885, Acc=0.6308
Epoch 14: Train Loss=0.6573, Acc=0.6837 | Val Loss=0.6784, Acc=0.6596
Epoch 15: Train Loss

...

Epoch 01: Train Loss=0.6907, Acc=0.5183 | Val Loss=0.6572, Acc=0.7077
Epoch 02: Train Loss=0.6742, Acc=0.7024 | Val Loss=0.6640, Acc=0.6538
Epoch 03: Train Loss=0.6713, Acc=0.6822 | Val Loss=0.6504, Acc=0.6885
Epoch 04: Train Loss=0.6707, Acc=0.6841 | Val Loss=0.6476, Acc=0.7308
Epoch 05: Train Loss=0.6756, Acc=0.6861 | Val Loss=0.6621, Acc=0.7538
Epoch 06: Train Loss=0.6692, Acc=0.7005 | Val Loss=0.6485, Acc=0.7327
Epoch 07: Train Loss=0.6656, Acc=0.6707 | Val Loss=0.6577, Acc=0.6615
Epoch 08: Train Loss=0.6631, Acc=0.6875 | Val Loss=0.6556, Acc=0.7154
Epoch 09: Train Loss=0.6673, Acc=0.6981 | Val Loss=0.6549, Acc=0.6846
Epoch 10: Train Loss=0.6680, Acc=0.7096 | Val Loss=0.6618, Acc=0.7038
Epoch 11: Train Loss=0.6637, Acc=0.6788 | Val Loss=0.6518, Acc=0.7308
Epoch 12: Train Loss=0.6494, Acc=0.7019 | Val Loss=0.6653, Acc=0.6635
Epoch 13: Train Loss=0.6508, Acc=0.7264 | Val Loss=0.6629, Acc=0.7635
Epoch 14: Train Loss=0.6507, Acc=0.7192 | Val Loss=0.6662, Acc=0.7269
Epoch 15: Train Loss

C:\Users\USER\anaconda3\envs\newenv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 01: Train Loss=0.6872, Acc=0.6178 | Val Loss=0.6815, Acc=0.7038
Epoch 02: Train Loss=0.6809, Acc=0.7029 | Val Loss=0.6778, Acc=0.6942
Epoch 03: Train Loss=0.6738, Acc=0.6966 | Val Loss=0.6759, Acc=0.6923
Epoch 04: Train Loss=0.6705, Acc=0.7236 | Val Loss=0.6756, Acc=0.6692
Epoch 05: Train Loss=0.6682, Acc=0.6880 | Val Loss=0.6770, Acc=0.6981
Epoch 06: Train Loss=0.6637, Acc=0.7067 | Val Loss=0.6726, Acc=0.6654
Epoch 07: Train Loss=0.6649, Acc=0.6764 | Val Loss=0.6702, Acc=0.6654
Epoch 08: Train Loss=0.6553, Acc=0.6918 | Val Loss=0.6695, Acc=0.6712
Epoch 09: Train Loss=0.6586, Acc=0.7091 | Val Loss=0.6743, Acc=0.6731
Epoch 10: Train Loss=0.6570, Acc=0.6990 | Val Loss=0.6740, Acc=0.6827
Epoch 11: Train Loss=0.6562, Acc=0.7154 | Val Loss=0.6774, Acc=0.7000
Epoch 12: Train Loss=0.6595, Acc=0.7130 | Val Loss=0.6727, Acc=0.6865
Epoch 13: Train Loss=0.6516, Acc=0.6923 | Val Loss=0.6783, Acc=0.6942
Epoch 14: Train Loss=0.6541, Acc=0.7288 | Val Loss=0.6763, Acc=0.6577
Epoch 15: Train Loss

C:\Users\USER\anaconda3\envs\newenv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 01: Train Loss=0.6954, Acc=0.3611 | Val Loss=0.6806, Acc=0.2519
Epoch 02: Train Loss=0.6874, Acc=0.4702 | Val Loss=0.6768, Acc=0.5615
Epoch 03: Train Loss=0.6756, Acc=0.5663 | Val Loss=0.6806, Acc=0.7788
Epoch 04: Train Loss=0.6773, Acc=0.6447 | Val Loss=0.6677, Acc=0.7404
Epoch 05: Train Loss=0.6767, Acc=0.6058 | Val Loss=0.6587, Acc=0.6962
Epoch 06: Train Loss=0.6749, Acc=0.6308 | Val Loss=0.6582, Acc=0.6981
Epoch 07: Train Loss=0.6735, Acc=0.6587 | Val Loss=0.6560, Acc=0.7096
Epoch 08: Train Loss=0.6725, Acc=0.6827 | Val Loss=0.6579, Acc=0.7135
Epoch 09: Train Loss=0.6695, Acc=0.6813 | Val Loss=0.6534, Acc=0.6750
Epoch 10: Train Loss=0.6618, Acc=0.6740 | Val Loss=0.6531, Acc=0.6846
Epoch 11: Train Loss=0.6602, Acc=0.6957 | Val Loss=0.6555, Acc=0.6962
Epoch 12: Train Loss=0.6583, Acc=0.7034 | Val Loss=0.6542, Acc=0.6788
Epoch 13: Train Loss=0.6533, Acc=0.7106 | Val Loss=0.6614, Acc=0.7385
Epoch 14: Train Loss=0.6418, Acc=0.7284 | Val Loss=0.6569, Acc=0.7000
Epoch 15: Train Loss